In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
def linreg(X, y, reg=0.0):
	'''
		X is matrix with dimension m x (n + 1).
		y is label with dimension m x 1.
		reg is the parameter for regularization.

		Return the optimal weight matrix.
	'''
	# Hint: Find the numerical solution for part c
	# Use np.eye to create identity matrix
	# Use np.linalg.solve to solve W_opt

	eye = np.eye(X.shape[1])
	eye[0, 0] = 0 # don't regularize the bias term
	W_opt = np.linalg.solve(X.T @ X + reg * eye, X.T @ y)
	return W_opt

def predict(W, X):
	'''
		W is a weight matrix with bias.
		X is the data with dimension m x (n + 1).

		Return the predicted label, y_pred.
	'''
	return X * W

def find_RMSE(W, X, y):
	'''
		W is the weight matrix with bias.
		X is the data with dimension m x (n + 1).
		y is label with dimension m x 1.

		Return the root mean-squared error.
	'''
	y_pred = predict(W, X)
	diff = y - y_pred
	m = X.shape[0]
	MSE = np.linalg.norm(diff, 2) ** 2 / m
	return np.sqrt(MSE)

def RMSE_vs_lambda(X_train, y_train, X_val, y_val):
	'''
		X is the data with dimension m x (n + 1).
		y is the label with dimension m x 1.

		Genearte a plot of RMSE vs lambda.
		Return the regularization parameter that minimizes RMSE.
	'''
	# Set up plot style
	plt.style.use('ggplot')

	RMSE_list = []
 
	# Construct a list of regularization parameters with random uniform sampling
	# Then, generate a list of W_opt's according to these parameters
	# Finally, generate a list of RMSE according to reg_list
	reg_list = np.random.uniform(0.0, 150.0, 150)
	reg_list.sort()
	W_list = [linreg(X_train, y_train, reg = lb) for lb in reg_list]
	for index in range(len(reg_list)):
		W_opt = W_list[index]
		RMSE_list.append(find_RMSE(W_opt, X_val, y_val))

	# Plot RMSE vs lambda
	RMSE_vs_lambda_plot, = plt.plot(reg_list, RMSE_list)
	plt.setp(RMSE_vs_lambda_plot, color = 'red')
	plt.title('RMSE vs lambda')
	plt.xlabel('lambda')
	plt.ylabel('RMSE')
	plt.savefig('RMSE_vs_lambda.png', format = 'png')
	plt.close()
	print('==> Plotting completed.')

	# Find the regularization value that minimizes RMSE
	opt_lambda_index = np.argmin(RMSE_list)
	reg_opt = reg_list[opt_lambda_index]
	return reg_opt

def norm_vs_lambda(X_train, y_train, X_val, y_val):
	'''
		X is the data with dimension m x (n + 1).
		y is the label with dimension m x 1.

		Genearte a plot of norm of the weights vs lambda.
	'''
	# You may reuse the code for RMSE_vs_lambda
	# to generate the list of weights and regularization parameters
	reg_list = np.random.uniform(0.0, 150.0, 150)
	reg_list.sort()
	W_list = [linreg(X_train, y_train, reg = lb) for lb in reg_list]

	# Calculate the norm of each weight
	norm_list = [np.linalg.norm(W, 2) for W in W_list]

	# Plot norm vs lambda
	norm_vs_lambda_plot, = plt.plot(reg_list, norm_list)
	plt.setp(norm_vs_lambda_plot, color = 'blue')
	plt.title('norm vs lambda')
	plt.xlabel('lambda')
	plt.ylabel('norm')
	plt.savefig('norm_vs_lambda.png', format = 'png')
	plt.close()
	print('==> Plotting completed.')

def linreg_no_bias(X, y, reg = 0.0):
	'''
		X is matrix with dimension m x n.
		y is label with dimension m x 1.
		reg is the parameter for regularization.

		Return the optimal weight and bias separately.
	'''
	t_start = time.time()
 
	# Find the numerical solution in part d
	m = X.shape[0]
	ones = np.eye(m)
	Aggregate = X.T @ (np.eye(m) - np.ones(m) / m)
	W_opt = np.linalg.solve(Aggregate @ X + reg * np.eye(Aggregate.shape[0]), \
		Aggregate @ y)
	b_opt = sum((y - X @ W_opt)) / m
 
	# Benchmark report
	t_end = time.time()
	print('--Time elapsed for training: {t:4.2f} seconds'.format(\
				t = t_end - t_start))
	return b_opt, W_opt

def grad_descent(X_train, y_train, X_val, y_val, reg = 0.0, \
	lr_W = 2.5e-12, lr_b = 0.2, max_iter = 150, eps = 1e-6, print_freq = 25):
	'''
		X is matrix with dimension m x n.
		y is label with dimension m x 1.
		reg is the parameter for regularization.
		lr_W is the learning rate for weights.
		lr_b is the learning rate for bias.
		max_iter is the maximum number of iterations.
		eps is the threshold of the norm for the gradients.
		print_freq is the frequency of printing the report.

		Return the optimal weight and bias by gradient descent.
	'''
	m_train, n = X_train.shape
	m_val = X_val.shape[0]
 
	# initialize the weights and bias and their corresponding gradients

	W = np.zeros((n, 1))
	b = 0.
	W_grad = np.ones_like(W)
	b_grad = 1.

	obj_train = []
	obj_val = []
	print('==> Running gradient descent...')
	iter_num = 0
	t_start = time.time()

	while np.linalg.norm(W_grad) > eps and np.linalg.norm(b_grad) > eps \
	and iter_num < max_iter:
		# calculate norms
		train_rmse = np.sqrt(np.linalg.norm((X_train @ W).reshape((-1, 1)) \
			+ b - y_train) ** 2 / m_train)
		obj_train.append(train_rmse)
		val_rmse = np.sqrt(np.linalg.norm((X_val @ W).reshape((-1, 1)) \
			+ b - y_val) ** 2 / m_val)
		obj_val.append(val_rmse)
		# calculate gradient
		W_grad = ((X_train.T @ X_train + reg * np.eye(n)) @ W \
			+ X_train.T @ (b - y_train)) / m_train
		b_grad = (sum(X_train @ W) - sum(y_train) + b * m_train) / m_train
		# update weights and bias
		W -= lr_W * W_grad
		b -= lr_b * b_grad
		# print statements
		if (iter_num + 1) % print_freq == 0:
			print('-- Iteration{} - training rmse {: 4.4f} - gradient norm {: 4.4E}'.format(\
				iter_num + 1, train_rmse, np.linalg.norm(W_grad)))
		iter_num += 1

	# Benchmark report
	t_end = time.time()
	print('--Time elapsed for training: {t:4.2f} seconds'.format(\
			t = t_end - t_start))
 
	# generate convergence plot
	train_rmse_plot, = plt.plot(range(iter_num), obj_train)
	plt.setp(train_rmse_plot, color = 'red')
	val_rmse_plot, = plt.plot(range(iter_num), obj_val)
	plt.setp(val_rmse_plot, color = 'green')
	plt.legend((train_rmse_plot, val_rmse_plot), \
		('Training RMSE', 'Validation RMSE'), loc = 'best')
	plt.title('RMSE vs iteration')
	plt.xlabel('iteration')
	plt.ylabel('RMSE')
	plt.savefig('convergence.png', format = 'png')
	plt.close()
	print('==> Plotting completed.')

	return b, W

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
train_pct = 2.0 / 3
val_pct = 5.0 / 6
df = pd.read_csv('https://math189sp19.github.io/data/online_news_popularity.csv', sep = ', ', engine = 'python')

In [4]:
df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [5]:
# split the data frame by type: training, validation, and test
df['type'] = ''
df.loc[:int(train_pct * len(df)), 'type'] = 'train'
df.loc[int(train_pct * len(df)) : int(val_pct * len(df)), 'type'] = 'val'
df.loc[int(val_pct * len(df)):, 'type'] = 'test'

In [6]:
# extracting columns into training, validation, and test data
X_train = np.array(df[df.type == 'train'][[col for col in df.columns \
    if col not in ['url', 'shares', 'type']]])
y_train = np.log(df[df.type == 'train'].shares).values.reshape((-1, 1))

X_val = np.array(df[df.type == 'val'][[col for col in df.columns \
    if col not in ['url', 'shares', 'type']]])
y_val = np.log(df[df.type == 'val'].shares).values.reshape((-1, 1))

X_test = np.array(df[df.type == 'test'][[col for col in df.columns \
    if col not in ['url', 'shares', 'type']]])
y_test = np.log(df[df.type == 'test'].shares).values.reshape((-1, 1))

In [7]:
# Stack a column of ones to the feature data
# Use np.ones / np.ones_like to create a column of ones
# Use np.hstack to stack the column to the matrix
X_train = np.hstack((np.ones_like(y_train), X_train))
X_val = np.hstack((np.ones_like(y_val), X_val))
X_test = np.hstack((np.ones_like(y_test), X_test))

In [8]:
# Convert data to matrix
X_train = np.matrix(X_train)
y_train = np.matrix(y_train)
X_val = np.matrix(X_val)
y_val = np.matrix(y_val)
X_test = np.matrix(X_test)
y_test = np.matrix(y_test)

In [9]:
# STEP 1: RMSE vs lambda
print('\n==> Step 1: RMSE vs lambda...')
# Fill in the code in linreg, findRMSE, and RMSE_vs_lambda
reg_opt = RMSE_vs_lambda(X_train, y_train, X_val, y_val)
print('==> The optimal regularization parameter is {reg: 4.4f}.'.format(\
    reg = reg_opt))


==> Step 1: RMSE vs lambda...
==> Plotting completed.
==> The optimal regularization parameter is  8.7286.


In [10]:
# Find the optimal weights and bias for future use in step 3
W_with_b_1 = linreg(X_train, y_train, reg = reg_opt)
b_opt_1 = W_with_b_1[0]
W_opt_1 = W_with_b_1[1: ]

In [11]:
# Report the RMSE with the found optimal weights on validation set
val_RMSE = find_RMSE(W_with_b_1, X_val, y_val)
print('==> The RMSE on the validation set with the optimal regularization parameter is {RMSE: 4.4f}.'.format(\
    RMSE=val_RMSE))

# Report the RMSE with the found optimal weights on test set
test_RMSE = find_RMSE(W_with_b_1, X_test, y_test)
print('==> The RMSE on the test set with the optimal regularization parameter is {RMSE: 4.4f}.'.format(\
    RMSE=test_RMSE))

# Norm vs lambda
norm_vs_lambda(X_train, y_train, X_val, y_val)

==> The RMSE on the validation set with the optimal regularization parameter is  0.8340.
==> The RMSE on the test set with the optimal regularization parameter is  0.8628.
==> Plotting completed.


In [12]:
# Part c
# From here on, we will strip the columns of ones for all data
X_train = X_train[:, 1:]
X_val = X_val[:, 1:]
X_test = X_test[:, 1:]
# Fill in the code in linreg_no_bias
# Compare the result with the one from step 1
# The difference in norm should be a small scalar (i.e, 1e-10)
print('\n==> Step 3: Linear regression without bias...')
b_opt_2, W_opt_2 = linreg_no_bias(X_train, y_train, reg = reg_opt)
diff_bias = np.linalg.norm(b_opt_2 - b_opt_1)
print('==> Difference in bias is {diff: 4.4E}'.format(diff = diff_bias))
diff_W = np.linalg.norm(W_opt_2 -W_opt_1)
print('==> Difference in weights is {diff: 4.4E}'.format(diff = diff_W))


==> Step 3: Linear regression without bias...


--Time elapsed for training: 4.12 seconds
==> Difference in bias is  2.4734E-10
==> Difference in weights is  3.6846E-10


In [13]:
# Part e
print('\n==> Step 4: Gradient descent')
b_gd, W_gd = grad_descent(X_train, y_train, X_val, y_val, reg = reg_opt)
# Compare the result from the one from step 1
diff_bias = np.linalg.norm(b_gd - b_opt_1)
print('==> Difference in bias is {diff: 4.4E}'.format(diff = diff_bias))
diff_W = np.linalg.norm(W_gd -W_opt_1)
print('==> Difference in weights is {diff: 4.4E}'.format(diff = diff_W))


==> Step 4: Gradient descent
==> Running gradient descent...
-- Iteration25 - training rmse  1.6604 - gradient norm  3.6435E+04
-- Iteration50 - training rmse  1.2519 - gradient norm  2.4237E+04
-- Iteration75 - training rmse  1.0664 - gradient norm  1.5202E+04
-- Iteration100 - training rmse  0.9896 - gradient norm  9.7337E+03
-- Iteration125 - training rmse  0.9596 - gradient norm  6.3025E+03
-- Iteration150 - training rmse  0.9481 - gradient norm  4.1295E+03
--Time elapsed for training: 24.07 seconds
==> Plotting completed.
==> Difference in bias is  1.5387E-01
==> Difference in weights is  7.9875E-01
